# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





In [66]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

# 打開瀏覽器
browser = webdriver.Chrome()
browser.get("https://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

In [67]:
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value("11")
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value("2018")

browser.find_element_by_id("ctl05_btnQuery").click()

### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [68]:
html = browser.page_source

soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', class_="TABLE_G")

d = {}
for i in table.find_all('tr')[1:9]:
    if len(i) == 7:
        name = i.find('td', class_='no-alt').text
        d.setdefault(name,{})
        date = i.find_all('td')[2].text
        value = i.find_all('td')[3].text
        d[name][date] = value
    else:
        date = i.find_all('td')[0].text
        value = i.find_all('td')[1].text
        d[name][date] = value

print(d)

{'SO2': {'2018/01': '1.80', '2018/02': '1.90', '2018/03': '2.20', '2018/04': '2.30', '2018/05': '3.10', '2018/06': '2.70', '2018/07': '2.20', '2018/08': '2.40'}}


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [75]:
dic = {}
for td in table.find_all('tr')[1:]:
    for i, t in enumerate(td.find_all('td')):
        try:
            if len(td) == 7:
                if i == 0:
                    name = t.text
                    if name == 'SO2' or name == 'CO':
                        dic.setdefault(name,{})
                elif i == 2:
                    date = t.text
                elif i == 3:
                    value = t.text
                    dic[name][date] = value
            else:
                if i == 0:
                    date = t.text
                elif i == 1:
                    value = t.text
                    if date < '2018/09':
                        dic[name][date] = value
        except:
            pass

print(dic)

{'SO2': {'2018/01': '1.80', '2018/02': '1.90', '2018/03': '2.20', '2018/04': '2.30', '2018/05': '3.10', '2018/06': '2.70', '2018/07': '2.20', '2018/08': '2.40'}, 'CO': {'2018/01': '0.34', '2018/02': '0.44', '2018/03': '0.40', '2018/04': '0.38', '2018/05': '0.34', '2018/06': '0.29', '2018/07': '0.21', '2018/08': '0.30'}}
